In [1]:
import os
from nbed.driver import NbedDriver
import numpy as np

from nbed.ham_builder import HamiltonianBuilder
from openfermion import get_sparse_operator
import scipy as sp
    
from pyscf import gto, scf, fci, cc

/home/lex/anaconda3/envs/embed/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# get xyz file for water
notebook_dir = os.getcwd()
NBed_dir = os.path.dirname(notebook_dir)
Test_dir = os.path.join(NBed_dir, 'tests')
mol_dir = os.path.join(Test_dir, 'molecules')

water_xyz_path = os.path.join(mol_dir, 'water.xyz')

In [3]:
geometry_path = os.path.join(mol_dir, 'water.xyz')
# geometry_path = os.path.join(mol_dir, 'H4.xyz')

In [4]:
with open(geometry_path, 'r') as infile:
    xyz_string = infile.read()
    
print(xyz_string)

3

O   0.00000  0.0000000  0.1653507
H   0.00000  0.7493682 -0.4424329
H   0.00000 -0.7493682 -0.4424329 


In [5]:
# options
geometry = xyz_string
n_active_atoms=1
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
run_virtual_localization = False

run_fci_emb = False
run_ccsd_emb = False
max_ram_memory = 8_000
_init_huzinaga_rhf_with_mu = False

max_hf_cycles=5000


projector = 'both'
localization = 'spade' # spade, ibo
occupied_threshold = 0.99
virtual_threshold = 0.95

In [6]:
driver = NbedDriver(geometry = geometry,
                    n_active_atoms=n_active_atoms,
                    basis = basis,
                    xc_functional = xc_functional,
                    run_virtual_localization = run_virtual_localization,
                    run_fci_emb = run_fci_emb,
                    run_ccsd_emb = run_ccsd_emb,
                    max_ram_memory = max_ram_memory,
                    _init_huzinaga_rhf_with_mu = _init_huzinaga_rhf_with_mu,
                    max_hf_cycles=max_hf_cycles,
                    occupied_threshold=occupied_threshold,
                    projector = projector,
                    localization = localization)

# FCI calc

In [7]:
charge=0
spin=0

full_system_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol.build()

global_rhf = scf.RHF(full_system_mol)
global_rhf.verbose=1
global_rhf.conv_tol = 1e-6
global_rhf.kernel()


## run FCI embedded
glob_FCI = fci.FCI(global_rhf)
glob_FCI.run()

print(f'full FCI energy: {glob_FCI.e_tot}')

full FCI energy: -75.01535247852486


# 1. Low-level whole system calculation

The first step is to run a DFT caluclation of the whole system.

In [8]:
### inputs

charge=0
spin=0

full_system_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol.build()

global_rks = scf.RKS(full_system_mol)
global_rks.verbose=1
global_rks.xc = xc_functional
global_rks.conv_tol = 1e-6
global_rks.kernel()

global_rks_Fock = global_rks.get_fock()
c_global_rks_full = global_rks.mo_coeff
global_rks_etot = global_rks.energy_tot()
global_rks.mo_energy = np.diag(c_global_rks_full.conj().T @ global_rks_Fock@ c_global_rks_full)
global_rks_mo_energies = global_rks.mo_energy 
global_rks_mo_energies

array([-18.83525318,  -0.93132671,  -0.43263415,  -0.23435814,
        -0.14179089,   0.35582625,   0.46190031])

In [9]:
# np.around(c_global_rks_full.conj().T @ global_rks_Fock@ c_global_rks_full, 5)

In [10]:
# checking global DFT calc
c_occ = global_rks.mo_coeff[:, global_rks.mo_occ>0]
dmat = 2* c_occ @ c_occ.T
e_tot = global_rks.energy_nuc() + global_rks.energy_elec(dm =dmat)[0]

np.isclose(global_rks.e_tot, e_tot)

True

In [11]:
# getting matrices manual
two_veff = global_rks.get_veff(dm=dmat)
vj = two_veff.vj
vk = two_veff.vk 

In [12]:
# calc xc contribution (DFT) via manual integral
ni = global_rks._numint
nelec, exc, vxc = ni.nr_rks(global_rks.mol, global_rks.grids, global_rks.xc, global_rks.make_rdm1())
exc - 0.25 * np.einsum('ij, ji ->', vk, dmat) 

-9.408825304427085

In [13]:
# check if this gives the same results
np.einsum('ij, ji ->',-0.25*vk, dmat) + exc

-9.408825304427085

In [14]:
two_veff.exc

-9.408825304427085

In [15]:
# two_veff construction
np.allclose(two_veff, vj - 0.5*vk + vxc)

True

In [16]:
print(two_veff.ecoul)
print(0.5 * np.einsum('ij, ji ->', vj, dmat) )

47.2318964211318
47.2318964211318


In [17]:
print(two_veff.exc + two_veff.ecoul)
print(0.5 * np.einsum('ij, ji ->', vj - 0.5*vk, dmat)+ exc)

37.82307111670472
37.82307111670472


In [18]:
## DFT fock matrix
F_obj = global_rks.get_fock()
F_manual = two_veff + global_rks.get_hcore()

np.allclose(F_obj, F_manual)

True

In [19]:
FF =  0.5*(vj - 0.5*vk) + global_rks.get_hcore()
np.einsum('ij, ji ->',FF, dmat) + exc

-84.40616856232964

In [20]:
np.allclose(two_veff, vj - 0.5*vk + vxc)

True

In [21]:
# subtract out the vxc part!!! (aka only leaving: 0.5*(vj - 0.5*vk) part!)

FF2 =  0.5*(two_veff - vxc)  + global_rks.get_hcore()
np.einsum('ij, ji ->',FF2, dmat) + exc

-84.40616856232964

In [22]:
global_rks.energy_elec()[0]

-84.40616856232964

# 2. Localize

We then use this object to find LOCALIZED systems (active and enviroment)

(more details given in notebook 2)

In [23]:
from nbed.localizers import (
    BOYSLocalizer,
    IBOLocalizer,
    Localizer,
    PMLocalizer,
    SPADELocalizer,
)

run_virtual_localization = False


localizers = {
    'boys':BOYSLocalizer,
    'ibo':IBOLocalizer,
    'pipek-mezey': PMLocalizer,
    'spade': SPADELocalizer
}

In [24]:
### inputs

charge=0
spin=0

full_system_mol2 = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol2.build()

global_rks2 = scf.RKS(full_system_mol2)
global_rks2.verbose=1
global_rks2.xc = xc_functional
global_rks2.conv_tol = 1e-6
global_rks2.kernel()

global_rks_Fock = global_rks2.get_fock()
c_global_rks_full = global_rks2.mo_coeff
global_rks_etot = global_rks2.energy_tot()
global_rks2.mo_energy = np.diag(c_global_rks_full.conj().T @ global_rks_Fock@ c_global_rks_full)

In [25]:
localized_system = localizers[localization](global_rks2,
                                        n_active_atoms,
                                        occ_cutoff=occupied_threshold,
                                        virt_cutoff=virtual_threshold,
                                        run_virtual_localization=run_virtual_localization,
)

In [26]:
print(f'active inds: {localized_system.active_MO_inds}')
print(f'enviro inds: {localized_system.enviro_MO_inds}')

active inds: [0 1 2]
enviro inds: [3 4]


# 3. Run subsystem DFT (uses localized rks)

In [27]:
def rks_components(
    RKS_obj, subsystem_dm: np.ndarray):
    """Calculate the components of subsystem energy from a RKS DFT calculation.

    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.

    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)

    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    dm_matrix = subsystem_dm
    # It seems that PySCF lumps J and K in the J array
    two_e_term = RKS_obj.get_veff(dm=dm_matrix)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", RKS_obj.get_hcore(), dm_matrix)
        + two_e_term.ecoul
        + two_e_term.exc
    )

    return energy_elec, e_xc, j_mat

In [28]:
(e_act, e_xc_act, j_act) = rks_components(global_rks, localized_system.dm_active)

(e_env, e_xc_env, j_env) = rks_components(global_rks, localized_system.dm_enviro)

# Computing cross subsystem terms
two_e_term_total = global_rks.get_veff(
    dm=localized_system.dm_active + localized_system.dm_enviro
)
e_xc_total = two_e_term_total.exc

j_cross = 0.5 * (
    np.einsum("ij,ij", localized_system.dm_active, j_env)
    + np.einsum("ij,ij", localized_system.dm_enviro, j_act)
)
# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - e_xc_act - e_xc_env

# overall two_electron cross energy
two_e_cross = j_cross + k_cross + xc_cross

energy_DFT_components = (e_act + e_env + two_e_cross + global_rks.energy_nuc())

if not np.isclose(energy_DFT_components, global_rks.e_tot):
    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT")


In [29]:
# e_act, e_env, two_e_cross, localized_system, j_env = subsystem_dft(localized_system, localized_system.rks)

# DFT energy made from component parts
E_dft_from_components = e_act + e_env + two_e_cross + global_rks.energy_nuc()
np.isclose(global_rks.e_tot, E_dft_from_components)

True

In [30]:
dm_full = global_rks.make_rdm1()

full = global_rks.get_veff(dm=dm_full)
# print(full.exc + full.ecoul)
A= global_rks.get_veff(dm=localized_system.dm_active)
B= global_rks.get_veff(dm=localized_system.dm_enviro)
non_add = (full.exc + full.ecoul) - (A.exc + A.ecoul + B.exc + B.ecoul)

np.isclose(two_e_cross, non_add)

True

In [31]:
A_energy = np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_active) + (A.exc + A.ecoul)
B_energy = np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_enviro) + (B.exc + B.ecoul)
components = A_energy + B_energy + non_add + global_rks.energy_nuc()
np.isclose(global_rks.e_tot, components)

True

In [32]:
j_active = A.vj
E_manual = np.einsum("ij,ji->", global_rks.get_hcore() + j_active/2, localized_system.dm_active) + A.exc
np.isclose(A_energy, E_manual)

True

In [33]:
EE = (np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_active) +
      np.einsum("ij,ji->", global_rks.get_hcore(), localized_system.dm_enviro) +
      full.exc + full.ecoul
      + global_rks.energy_nuc()
    )
np.isclose(global_rks.e_tot, EE)

True

# 4. Get DFT potential

In [34]:
g_act_and_env = global_rks.get_veff(
    dm=(localized_system.dm_active + localized_system.dm_enviro)
)
g_act = global_rks.get_veff(dm=localized_system.dm_active)
dft_potential = g_act_and_env - g_act # + j_env

In [35]:
# localized_system.rks
dmat = 2* c_occ @ c_occ.T
print(np.allclose(dmat, localized_system.dm_active + localized_system.dm_enviro))

print(np.allclose(g_act_and_env, two_veff))

True
True


In [36]:
# vj_act = g_act.vj
# vk_act= g_act.vk

# ni = global_rks._numint
# nelec_act, exc_act, vxc_act = ni.nr_rks(global_rks.mol,
#                                         global_rks.grids, 
#                                         global_rks.xc,
#                                         localized_system.dm_active)


# np.allclose(g_act, (vj_act - 0.5*vk_act) + vxc_act)

dft_potential is

$$DFT\: potential = g[\gamma_{act} + \gamma_{env}] - g[\gamma_{act}]$$

In [37]:
rks_emb_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
rks_emb_mol.nelectron = 2 * len(localized_system.active_MO_inds) # <--change no e-
rks_emb_mol.build()

rks_emb = scf.RKS(rks_emb_mol)
rks_emb.verbose=1
rks_emb.xc = xc_functional
rks_emb.conv_tol = 1e-6

s_mat = global_rks.get_ovlp()
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat

h_core_Std = rks_emb.get_hcore()
rks_emb.get_hcore = lambda *args: h_core_Std + 1e6*enviro_projector + dft_potential

rks_emb.kernel()

-51.990665456631255

In [38]:
y_emb =  rks_emb.make_rdm1()
diff_gamma = y_emb - localized_system.dm_active

In [39]:
e_act

-77.37494139936759

In [40]:
(global_rks.energy_elec(dm=y_emb)[0]  
 + np.einsum("ij,ij->",  dft_potential+1e6*enviro_projector, diff_gamma) 
 )

-77.37494171922

In [41]:
(rks_emb_mol.energy_elec(dm=y_emb)[0]  
 + np.einsum("ij,ij->",  dft_potential+1e6*enviro_projector, diff_gamma) 
 )

converged SCF energy = -69.6107725130686


-77.21517943251548

In [42]:
asdf

NameError: name 'asdf' is not defined

# WF approach

In [ ]:
charge=0
spin=0

full_system_mol = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol.nelectron = 2 * len(localized_system.active_MO_inds) # <- change number of e-
full_system_mol.build()

# HARTREE FOCK!
scf_method = scf.RHF(full_system_mol)
scf_method.verbose=1
scf_method.conv_tol = 1e-6

In [ ]:
# scf_method = init_localised_RHF_obj(geometry, basis, charge, localized_system)
mu_level_shift = 1e6

# redefine core H
std_hcore = scf_method.get_hcore()
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat
scf_method.get_hcore = lambda *args: std_hcore + mu_level_shift*enviro_projector + dft_potential

scf_method.kernel()

In [ ]:
correction = np.einsum("ij,ji", mu_level_shift*enviro_projector + dft_potential,
                       localized_system.dm_active) # <--- different 
correction

In [ ]:
scf_method.energy_elec()[0] - correction

In [ ]:
e_act

In [ ]:
## run FCI embedded
fci_scf_MU_embedded = fci.FCI(scf_method)

fci_scf_MU_embedded.run()

fci_scf_MU_embedded.e_tot

In [ ]:
e_corr = fci_scf_MU_embedded.e_tot - scf_method.e_tot
e_corr

In [ ]:
E_emb_act = scf_method.energy_elec()[0] - correction + e_corr 
E_emb_full = E_emb_act + e_env + two_e_cross + global_rks.energy_nuc()
E_emb_full

In [ ]:
print(f'global DFT energy: {global_rks.e_tot}')
print(f'global FCI energy: {glob_FCI.e_tot}')
print()

print(f'emb error : {abs(E_emb_full - glob_FCI.e_tot)}')
print(f'DFT error : {abs(global_rks.e_tot - glob_FCI.e_tot)}')

In [ ]:
charge=0
spin=0

full_system_mol_HUZ = gto.Mole(atom= geometry[2:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_HUZ.nelectron = 2 * len(localized_system.active_MO_inds) # <- change number of e-
full_system_mol_HUZ.build()

# HARTREE FOCK!
scf_method_HUZ = scf.RHF(full_system_mol_HUZ)
scf_method_HUZ.verbose=1
scf_method_HUZ.conv_tol = 1e-6

In [ ]:
from nbed.scf import huzinaga_RHF, huzinaga_RKS

In [ ]:
(c_active_embedded, 
 mo_energy, 
 dm_active_embedded,
 huzinaga_op_std,
 huz_rhf_conv_flag) =huzinaga_RHF(
    scf_method_HUZ,
    dft_potential,
    localized_system.dm_enviro,
    dm_conv_tol=1e-6,
    dm_initial_guess=None,
    use_DIIS= False,
)

hcore_std = scf_method_HUZ.get_hcore()
v_emb = huzinaga_op_std + dft_potential
scf_method_HUZ.get_hcore = lambda *args: hcore_std + v_emb
scf_method_HUZ.mo_coeff = c_active_embedded
scf_method_HUZ.mo_occ = scf_method_HUZ.get_occ(
    mo_energy, c_active_embedded
)
scf_method_HUZ.mo_energy = mo_energy
scf_method_HUZ.conv_check = huz_rhf_conv_flag

scf_method_HUZ.e_tot = scf_method_HUZ.energy_tot(dm=dm_active_embedded)


In [ ]:
mo_energy

In [ ]:
overlap = np.einsum('ij, ki -> i', c_active_embedded.T,  huzinaga_op_std @ c_active_embedded)
overlap

In [ ]:
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat
overlap = np.einsum('ij, ki -> i', c_active_embedded.T,  enviro_projector @ c_active_embedded)

overlap_by_size = overlap.argsort()[::-1]

n_env_mo = len(localized_system.enviro_MO_inds)
frozen_enviro_orb_inds = overlap_by_size[:n_env_mo]
active_MOs_occ_and_virt_embedded = [
                mo_i
                for mo_i in range(c_active_embedded.shape[1])
                if mo_i not in frozen_enviro_orb_inds
            ]

c_selected = c_active_embedded[:, active_MOs_occ_and_virt_embedded]
mo_energy_act = np.diag(c_selected.conj().T @ scf_method_HUZ.get_fock()@ c_selected)
mo_energy_act

In [ ]:
np.around(c_selected.conj().T @ scf_method_HUZ.get_fock()@ c_selected, 3)

In [ ]:
print(scf_method.mo_energy)

In [ ]:
## run FCI embedded
scf_method_HUZ.mo_coeff = c_active_embedded[:, active_MOs_occ_and_virt_embedded]

fci_scf_HUZ_embedded = fci.FCI(scf_method_HUZ)

fci_scf_HUZ_embedded.run()
fci_scf_HUZ_embedded.e_tot

In [ ]:
e_corr_HUZ = fci_scf_HUZ_embedded.e_tot - scf_method_HUZ.e_tot
e_corr_HUZ

In [ ]:
correction_HUZ = np.einsum("ij,ji", v_emb,
                       localized_system.dm_active) # <--- different 
correction_HUZ

In [ ]:
E_emb_act_HUZ = scf_method_HUZ.energy_elec(dm=dm_active_embedded)[0] - correction + e_corr_HUZ 
E_emb_full_HUZ = E_emb_act_HUZ + e_env + two_e_cross + global_rks.energy_nuc()


print(f'emb error : {abs(E_emb_full_HUZ - glob_FCI.e_tot)}')
print(f'DFT error : {abs(global_rks.e_tot - glob_FCI.e_tot)}')

In [ ]:
print(f'HUZ vs MU : {abs(E_emb_full - E_emb_full_HUZ)}')

# Looking at localized system

In [ ]:
print(np.around(localized_system.c_enviro, 3))

In [ ]:
enviro_projector = s_mat@ localized_system.dm_enviro @ s_mat

print(np.around(enviro_projector@ localized_system.c_loc_occ_and_virt, 3))

In [ ]:
proj_env = localized_system.dm_enviro/2 @ s_mat

print(np.around(localized_system.c_enviro, 3))
print()
print(np.around(proj_env @ localized_system.c_loc_occ_and_virt, 3))


In [ ]:
proj_env2 = s_mat @ localized_system.dm_enviro/2 # different order!
np.allclose(proj_env.conj().T, proj_env2)

# Get projector onto enviroment

In [ ]:
s_mat = global_rks.get_ovlp()
s_half = sp.linalg.fractional_matrix_power(s_mat, 0.5)
s_neg_half = sp.linalg.fractional_matrix_power(s_mat, -0.5)

# 1. Get orthogonal C matrix (localized)
c_loc_ortho = s_half @ localized_system.c_loc_occ_and_virt

# 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
#    (do this in orthongoal basis!)
#    note we only take MO environment indices!
ortho_proj = np.einsum(
    "ik,jk->ij",
    c_loc_ortho[:, localized_system.enviro_MO_inds],
    c_loc_ortho[:, localized_system.enviro_MO_inds],
)

In [ ]:
# note no two times for this dm!!!
dm = localized_system.c_enviro @ localized_system.c_enviro.T
ortho_proj2 =  s_half @ dm @ s_half
np.allclose(ortho_proj, ortho_proj2)

In [ ]:
enviro_projector = s_neg_half @ ortho_proj @ s_half
print(np.around(enviro_projector@localized_system.c_enviro, 5))
print()

enviro_projector = dm @ s_mat
print(np.around(enviro_projector@localized_system.c_enviro, 5))
print()

print(np.around(localized_system.c_enviro, 5))

In [ ]:
enviro_projector = s_mat@ dm @ s_mat # paper definition!
print(np.around(enviro_projector@localized_system.c_active, 10))

In [ ]:
np.trace(s_mat@ localized_system.dm_enviro) # @ s_mat)

In [ ]:
proj_act = localized_system.dm_active/2 @ s_mat # uses active dmat!

print(np.around(localized_system.c_active, 3))
print()
print(np.around(proj_act @ localized_system.c_loc_occ_and_virt, 3))